In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr

import eradiate

eradiate.config.settings["PROGRESS"] = "kernel"
from eradiate.constants import EARTH_RADIUS
from eradiate.scenes.surface import mesh_from_dem
from eradiate.units import unit_registry as ureg

sns.set_theme(style="ticks")
eradiate.set_mode("mono")

In [ ]:
# ds = xr.open_dataset("dem/gebco_2024/GEBCO_2024.nc")
ds = xr.open_dataset("dem/gebco_2024/gebco_2024_n50.0_s46.0_w-6.0_e0.0.nc")
# display(ds)

lon_bounds = [-5.255643817623447, -1.854804111820785]
lat_bounds = [46.98286701217718, 49.02004225968674]

lon = ds.lon.where((ds.lon > lon_bounds[0]) & (ds.lon < lon_bounds[1])).dropna(
    dim="lon"
)
lat = ds.lat.where((ds.lat > lat_bounds[0]) & (ds.lat < lat_bounds[1])).dropna(
    dim="lat"
)

brittany = ds.sel(lon=lon, lat=lat)

# lon_fine = np.linspace(lon.min(), lon.max(), 3333)
# lat_fine = np.linspace(lat.min(), lat.max(), 3333)
# brittany = brittany.interp(lon=lon_fine, lat=lat_fine)

brittany.elevation.plot.imshow()
display(brittany)

# Plane-parallel setup

In [ ]:
da = brittany.elevation.copy()
da *= 20
da[{"lon": (da.lon == da.lon.min()) | (da.lon == da.lon.max())}] = 0.0
da[{"lat": (da.lat == da.lat.min()) | (da.lat == da.lat.max())}] = 0.0
da.lon.attrs["units"] = "deg"
da.lat.attrs["units"] = "deg"
mesh, xlim, ylim = mesh_from_dem(da, geometry="plane_parallel", add_texcoords=True)
print(f"{(xlim, ylim) = }")

if not Path("brittany_pp.ply").exists():
    mi_mesh = mesh.instance
    mi_mesh.write_ply("brittany_pp.ply")

mesh_ply = eradiate.scenes.shapes.FileMeshShape(id="dem", filename="brittany_pp.ply")

In [ ]:
exp = eradiate.experiments.DEMExperiment(
    geometry="plane_parallel",
    illumination={
        "type": "directional",
        "zenith": 45.0,
    },
    atmosphere=None,
    # atmosphere={"type": "molecular"},
    surface={
        "type": "dem",
        "construct": "from_mesh",
        "mesh": mesh_ply,
        "xlon_lim": xlim,
        "ylat_lim": ylim,
        # "bsdf_mesh": {"type": "lambertian", "reflectance": 0.5},
        "bsdf_mesh": {"type": "checkerboard"},
        "bsdf_background": {"type": "lambertian", "reflectance": 0.3},
    },
    measures={
        "type": "perspective",
        # "origin": [0, -500, 100] * ureg.km,
        "origin": [0, 0, 500] * ureg.km,
        "target": [0, 0, 0],
        # "up": [0, 0, 1],
        "up": [0, 1, 0],
        "film_resolution": (640, 480),
    },
)
result = eradiate.run(exp)
plt.imshow(result.radiance.squeeze(), aspect="equal")
plt.axis("off")
# plt.colorbar(shrink=0.8)
plt.show()
plt.close()

# Spherical-shell setup

In [ ]:
da = brittany.elevation.copy()
da *= 20
da[{"lon": (da.lon == da.lon.min()) | (da.lon == da.lon.max())}] = 0.0
da[{"lat": (da.lat == da.lat.min()) | (da.lat == da.lat.max())}] = 0.0
da.lon.attrs["units"] = "deg"
da.lat.attrs["units"] = "deg"
mesh, lonlim, latlim = mesh_from_dem(da, geometry="spherical_shell", add_texcoords=True)

if not Path("brittany_ss.ply").exists():
    mi_mesh = mesh.instance
    mi_mesh.write_ply("brittany_ss.ply")

mesh_ply = eradiate.scenes.shapes.FileMeshShape(id="dem", filename="brittany_ss.ply")

In [ ]:
origin = [0, 0, EARTH_RADIUS.m_as("km")] * ureg.km

exp = eradiate.experiments.DEMExperiment(
    geometry="spherical_shell",
    illumination={
        "type": "directional",
        "zenith": 45.0,
    },
    atmosphere=None,
    # atmosphere={"type": "molecular"},
    surface={
        "type": "dem",
        "construct": "from_mesh",
        "geometry": "spherical_shell",
        "mesh": mesh_ply,
        "xlon_lim": lonlim,
        "ylat_lim": latlim,
        # "bsdf_mesh": {"type": "lambertian", "reflectance": 0.5},
        "bsdf_mesh": {"type": "checkerboard"},
        "bsdf_background": {"type": "lambertian", "reflectance": 0.3},
    },
    measures={
        "type": "perspective",
        # "origin": origin + [0, -500, 100] * ureg.km,
        "origin": origin + [0, 0, 500] * ureg.km,
        "target": origin,
        # "up": [0, 0, 1],
        "up": [0, 1, 0],
        "film_resolution": (640, 480),
    },
)
result = eradiate.run(exp)
plt.imshow(result.radiance.squeeze(), aspect="equal")
plt.axis("off")
# plt.colorbar(shrink=0.8)
plt.show()
plt.close()